In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import (
    precision_recall_fscore_support,
    roc_auc_score, average_precision_score,
    matthews_corrcoef, balanced_accuracy_score
)
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv1D, BatchNormalization, ReLU,
                                     MaxPooling1D, Dropout, Bidirectional, LSTM,
                                     Dense, Multiply, Softmax, Lambda)
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

In [16]:
X_train = pd.read_csv("../NN Datasets/x_train.csv")
y_train = pd.read_csv("../NN Datasets/y_train.csv")
X_train = X_train.drop([ X_train.columns[0], "Init_Win_bytes_forward" ], axis=1)
y_train = y_train.drop(y_train.columns[0], axis=1)

X_val = pd.read_csv("..//NN Datasets//x_val.csv")
y_val = pd.read_csv("..//NN Datasets//y_val.csv")
X_val = X_val.drop([X_val.columns[0], 'Init_Win_bytes_forward'] , axis=1)
y_val = y_val.drop(y_val.columns[0], axis=1)

X_test = pd.read_csv("../NN Datasets/x_test.csv")
y_test = pd.read_csv("../NN Datasets/y_test.csv")
X_test = X_test.drop([ X_test.columns[0], "Init_Win_bytes_forward" ], axis=1)
y_test = y_test.drop(y_test.columns[0], axis=1)

KeyboardInterrupt: 

In [ ]:
window_size = 20
num_features = 34
num_classes = 7

In [ ]:
y_val['label_code'].unique()

array([0, 1, 2, 3, 5, 4, 6])

In [ ]:
def prepare_windowed_data(X, y, window_size, num_classes):
    # 1) Конвертуємо в numpy
    X_arr = X.values if hasattr(X, 'values') else np.asarray(X)
    y_arr = y.values.squeeze() if hasattr(y, 'values') else np.asarray(y).squeeze()
    
    # 2) Визначаємо, скільки повних вікон вміститься
    n_rows, n_features = X_arr.shape
    n_windows = n_rows // window_size
    n_use = n_windows * window_size
    if n_use != n_rows:
        print(f"– Обрізаю {n_rows - n_use} рядків (залишаю {n_use} для {n_windows} вікон)")
    
    # 3) Обрізка
    X_trim = X_arr[:n_use]
    y_trim = y_arr[:n_use]
    
    # 4) reshape X → (n_windows, window_size, n_features)
    X_windows = X_trim.reshape(n_windows, window_size, n_features)
    
    # 5) reshape y → (n_windows, window_size) й беремо останню мітку вікна
    y_mat = y_trim.reshape(n_windows, window_size)
    y_last = y_mat[:, -1]
    
    # 6) one-hot
    y_windows = to_categorical(y_last, num_classes)
    
    return X_windows, y_windows

In [ ]:
window_size = 20
num_classes = 7

X_train, y_train = prepare_windowed_data(X_train, y_train, window_size, num_classes)
X_val,   y_val   = prepare_windowed_data(X_val,   y_val,   window_size, num_classes)
X_test,  y_test  = prepare_windowed_data(X_test,  y_test,  window_size, num_classes)

– Обрізаю 8 рядків (залишаю 11878320 для 593916 вікон)
– Обрізаю 5 рядків (залишаю 287580 для 14379 вікон)
– Обрізаю 18 рядків (залишаю 422900 для 21145 вікон)


In [ ]:
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")

X_train: (593916, 20, 34)
X_val: (14379, 20, 34)
X_test: (21145, 20, 34)


In [ ]:
def create_model(
    filters=64,
    kernel_size=3,
    pool_size=2,
    lstm_units=128,
    lstm_layers=1,
    dropout_rate=0.3,
    recurrent_dropout=0.1,
    activation='relu',
    kernel_initializer='he_uniform',
    optimizer='adam',
    optimizer__learning_rate=1e-3
):
    inp = Input(shape=(window_size, num_features))
    x = inp
    # 1D-CNN block
    x = Conv1D(filters, kernel_size, padding='same',
               kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv1D(filters, kernel_size+2, padding='same',
               kernel_initializer=kernel_initializer)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling1D(pool_size)(x)
    x = Dropout(dropout_rate)(x)

    # BiLSTM block
    for _ in range(lstm_layers):
        x = Bidirectional(LSTM(lstm_units,
                               return_sequences=True,
                               dropout=dropout_rate,
                               recurrent_dropout=recurrent_dropout))(x)
    x = Dropout(dropout_rate)(x)

    # Attention mechanism
    attn_scores = Dense(1, activation='tanh')(x)
    attn_scores = Softmax(axis=1)(attn_scores)
    context = Multiply()([x, attn_scores])
    context = Lambda(lambda z: K.sum(z, axis=1))(context)
    # Classification head
    x = Dense(128, activation=activation,
              kernel_initializer=kernel_initializer)(context)
    x = Dropout(dropout_rate)(x)
    out = Dense(num_classes, activation='softmax')(x)

    model = Model(inp, out)

    # Підбираємо оптимізатор
    opt = (optimizer(learning_rate=optimizer__learning_rate)
           if isinstance(optimizer, type)
           else optimizer)

    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
param_grid = {
    'filters':      [32, 64, 128],
    'kernel_size':  [3, 5, 7],
    'pool_size':    [2, 3],
    'lstm_units':   [64, 128, 256],
    'lstm_layers':  [1, 2],
    'dropout_rate': [0.2, 0.3, 0.5],
    'recurrent_dropout': [0.1, 0.2],
    'activation':   ['relu', 'selu'],
    'kernel_initializer': ['he_uniform', 'glorot_uniform'],
    'optimizer':           [tf.keras.optimizers.Adam, tf.keras.optimizers.RMSprop],
    'optimizer__learning_rate': [1e-2, 1e-3, 1e-4],
    'batch_size':          [64, 128, 256],
    'epochs':              [10, 20, 50]
}

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
]

In [ ]:
results = []

for params in ParameterGrid(param_grid):
    print('Training with params:', params)

    model_kwargs = {k:v for k,v in params.items()
                    if k not in ('batch_size','epochs')}
                                 
    model = create_model(**model_kwargs)
    # Навчання на train з валідацією на val
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        verbose=1,
        callbacks=callbacks
    )

    y_test_prob = model.predict(X_test)
    y_test_pred = np.argmax(y_test_prob, axis=1)

    test_acc     = np.mean(y_test_pred == y_test)
    test_bal_acc = balanced_accuracy_score(y_test, y_test_pred)
    test_mcc     = matthews_corrcoef(y_test, y_test_pred)
    test_roc_auc = roc_auc_score(y_test, y_test_prob,
                                 multi_class='ovo', average='macro')
    test_avg_prec= average_precision_score(y_test, y_test_prob, average='macro')
    
    # Збереження результатів
    row = params.copy()
    row.update({
        'test_accuracy': test_acc,
        'test_balanced_accuracy': test_bal_acc,
        'test_mcc': test_mcc,
        'test_roc_auc': test_roc_auc,
        'test_average_precision': test_avg_prec
    })
    results.append(row)

In [ ]:
df_results = pd.DataFrame(results)
print(df_results)

df_results.to_csv('grid_search_val_test_metrics.csv', index=False)